In [ ]:
import torch as th
from ActiveCritic.learner.active_critic_learner import ActiveCriticLearner
from ActiveCritic.learner.active_critic_args import ActiveCriticLearnerArgs
from ActiveCritic.tests.test_utils.utils import setup_ac_reach
from ActiveCritic.utils.gym_utils import make_dummy_vec_env, parse_sampled_transitions, sample_expert_transitions, DummyExtractor, new_epoch_reach
from ActiveCritic.utils.pytorch_utils import make_part_obs_data
from ActiveCritic.utils.dataset import DatasetAC

In [ ]:
device = 'cpu'

In [ ]:
ac, acps, env = setup_ac_reach()

In [ ]:
acla = ActiveCriticLearnerArgs()
acla.data_path = '/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/'
acla.device = device
acla.extractor = DummyExtractor()
acla.imitation_phase = False
acla.logname = 'test_acl'
acla.tboard = False

In [ ]:
acl = ActiveCriticLearner(ac_policy=ac, env=env, network_args_obj=acla)

In [55]:
epsiodes = 2
seq_len = 5
env, expert = make_dummy_vec_env(name='reach', seq_len=seq_len)
transitions = sample_expert_transitions(policy=expert.predict, env=env, episodes=epsiodes)
actions, observations, rewards = parse_sampled_transitions(transitions=transitions, new_epoch=new_epoch_reach, extractor=DummyExtractor())

Sampling expert transitions. 2


In [57]:
acts, obsv, rews = make_part_obs_data(actions=actions, observations=observations, rewards=rewards)

In [59]:
assert list(acts.shape) == [epsiodes*seq_len, seq_len, env.action_space.shape[0]]
assert list(obsv.shape) == [epsiodes*seq_len, seq_len, env.observation_space.shape[0]]
assert list(rews.shape) == [epsiodes*seq_len, seq_len]

In [60]:
for i in range(seq_len*epsiodes):
    assert th.equal(acts[i], actions[int(i/seq_len)])

In [66]:
for i in range(seq_len*epsiodes):
    org_index = int(i/seq_len)
    part_obs = th.clone(observations[org_index])
    part_obs[i%seq_len + 1 :] = 0
    assert th.equal(obsv[i], part_obs)
    assert th.equal(acts[i], actions[org_index])


tensor([[0.1411, 0.1430, 0.1467, 0.1523, 0.1597],
        [0.1411, 0.1430, 0.1467, 0.1523, 0.1597],
        [0.1411, 0.1430, 0.1467, 0.1523, 0.1597],
        [0.1411, 0.1430, 0.1467, 0.1523, 0.1597],
        [0.1411, 0.1430, 0.1467, 0.1523, 0.1597],
        [0.1350, 0.1376, 0.1425, 0.1497, 0.1591],
        [0.1350, 0.1376, 0.1425, 0.1497, 0.1591],
        [0.1350, 0.1376, 0.1425, 0.1497, 0.1591],
        [0.1350, 0.1376, 0.1425, 0.1497, 0.1591],
        [0.1350, 0.1376, 0.1425, 0.1497, 0.1591]])